# Coursera Applied Data Science Capstone Project

#### Applied DS: Week3 Assignment

In [21]:
#Import libraries
import pandas as pd
import numpy as np
import re
print('success')

success


In [22]:
#Load Data into dataframe
df1 =pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
print(df1.shape)
df1.head(2)

(20, 9)


,0,1,2,3,4,5,6,7,8
0,M1ANot assigned,M2ANot assigned,M3ANorth York(Parkwoods),M4ANorth York(Victoria Village),M5ADowntown Toronto(Regent Park / Harbourfront),M6ANorth York(Lawrence Manor / Lawrence Heights),M7AQueen's Park(Ontario Provincial Government),M8ANot assigned,M9AEtobicoke(Islington Avenue)
1,M1BScarborough(Malvern / Rouge),M2BNot assigned,M3BNorth York(Don Mills)North,M4BEast York(Parkview Hill / Woodbine Gardens),"M5BDowntown Toronto(Garden District, Ryerson)",M6BNorth York(Glencairn),M7BNot assigned,M8BNot assigned,M9BEtobicoke(West Deane Park / Princess Garden...


In [23]:
#Feature Engineering

df1_T = df1.transpose()

cols = df1.columns.values
print(cols,'\n')

lst_PostalCode = []
lst_Neigh = []
lst_Borough = []
# lst_temp = []
i=1

for c in range(len(cols)):
        lst_temp = []
        lst_temp.append(df1_T.iloc[c])
        for t in lst_temp:
            for j in range(len(t)):
                lst_PostalCode.append(t[j][:3])
                txt1 = re.search(r"\((.*)\)",t[j])
                if txt1:
                    lst_Neigh.append(txt1.group(1))
                else:
                    lst_Neigh.append('Not assigned')
                txt2 = re.search(r"(.*)\(",t[j][3:])
                if txt2:
                    lst_Borough.append(txt2.group(1))
                else:
                    lst_Borough.append('Not assigned')
                
                
print(len(lst_PostalCode))
print(len(lst_Neigh))
print(len(lst_Borough))

[0 1 2 3 4 5 6 7 8] 

180
180
180


In [24]:
#Creating DataFrame from list 
fin_df = pd.DataFrame(list(zip(lst_PostalCode,lst_Borough,lst_Neigh)),columns=['Postal Code','Borough','Neighborhood'])
fin_df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,Malvern / Rouge
2,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
3,M1E,Scarborough,Guildwood / Morningside / West Hill
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park
8,M1L,Scarborough,Golden Mile / Clairlea / Oakridge
9,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West


In [25]:
#replacing forward slash with comma 
fin_df=fin_df.apply(lambda x: x.str.replace('/',','))
print(fin_df[fin_df['Postal Code']=='M5A'])
print(fin_df.shape)
fin_df.head()

   Postal Code           Borough                Neighborhood
80         M5A  Downtown Toronto  Regent Park , Harbourfront
(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern , Rouge"
2,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
3,M1E,Scarborough,"Guildwood , Morningside , West Hill"
4,M1G,Scarborough,Woburn


In [26]:
#counting 'Not assigned' Borough
print(fin_df[fin_df.Borough=='Not assigned']['Borough'].count())
print(fin_df[fin_df.Borough!='Not assigned']['Borough'].count())
print(fin_df.shape)

77
103
(180, 3)


In [27]:
#we are only considering data which has assigned Borough values.
fin_df = fin_df[fin_df.Borough!='Not assigned']
print(fin_df.shape)

(103, 3)


In [28]:
#Combine all neighbourhoods where postcode and Borough are the same
fin_df = fin_df.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

In [29]:
#if Neighbourhood is not assigned, replace by borough name
for index, row in fin_df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        row['Neighborhood'] = row['Borough']

# define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood'] 

fin_df.columns = column_names

fin_df.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [30]:
fin_df.shape

(103, 3)

In [31]:
# pip install geocoder

In [32]:
df_latlng = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
df_latlng.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
print(fin_df.dtypes)
print(df_latlng.dtypes)

Postal Code     object
Borough         object
Neighborhood    object
dtype: object
Postal Code     object
Latitude       float64
Longitude      float64
dtype: object


In [39]:
fin_df = fin_df.merge(df_latlng,on='Postal Code',how='outer')
fin_df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
